# Run this notebook 
include complete calendar and public holidays

In [0]:
%pip install lightgbm

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)

In [0]:
%run /Users/ebizindavyi@delhaize.be/Data_preparation/Parse_xls_files

In [0]:
%run /Users/ebizindavyi@delhaize.be/Evaluation/Evaluation_functions

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
import lightgbm as lgb
from reusable.featurestore.features_schoolholidays import school_holidays
from pyspark.sql import functions as f
import holidays



## Evaluation Params

In [0]:
actual_cols = ['Dry Actuals', 'Dry Actuals']
fc_cols = ['Dry Fc', 'LGBM_FC']
error_suffixes = ['Dry_WOW', 'Dry_LGBM']
error_to_calc = ["Bias", "AbsError", "SqError","RelAbsError"]

## Load inbound data

In [0]:
inbound_2022_dir = '/dbfs/mnt/dataplatform/acc/DataScience/sandbox/ebiz/Inbound_FC/data/input/Ecom-Ops-status_2022.xlsx'

In [0]:
inbound_2022_df = create_master_df(inbound_2022_dir)


In [0]:
calendar_2022 = pd.DataFrame({'Dates':pd.date_range(dt.datetime(2022,1,3), periods=363, freq='d')})
inbound_2022_complete = calendar_2022.merge(inbound_2022_df[['Dates','Dry Actuals']], on='Dates',how='left').fillna(0).copy()

## LightGBM


##Parameters

In [0]:
#lgbm parameter
backtest_start_date = pd.to_datetime('2022-01-03')
backtest_end_date =  pd.to_datetime('2022-12-31')

#shift periods to create features(lags)
shift_period_1 = 7
shift_period_2 = 14
shift_period_3 = 21

# Initialize lgbm parameters 
params = {
    'objective': 'regression',  
    'metric': 'rmse',           
    'boosting_type': 'gbdt',    
    # 'max_depth':2,      
}
# variables 
feature_col = ['x_7days', 'x_14days', 'x_21days']
pred_period=6
end_horizon = 7
fc_freq=7
training_period= 180

In [0]:
inbound_2022_dry=inbound_2022_complete.copy()
shift_periods = [shift_period_1, shift_period_2, shift_period_3]
column_names = ['x_7days', 'x_14days', 'x_21days']

for i, period in enumerate(shift_periods):
    inbound_2022_dry[column_names[i]] = inbound_2022_dry['Dry Actuals'].shift(period)

In [0]:
def get_weekdays(inbound_df: pd.DataFrame, col_dates: str) -> pd.DataFrame:
    """
    Get weekdays as integers (Monday=0, Tuesday=1, ..., Sunday=6) and add them as a new column.
    Args:
    -input DataFrame.
    -The name of the date column.

    Returns:
    - DataFrame with a new column of weekday integers.
    """
    # Calculate weekday integers (Monday=0, Tuesday=1, ..., Sunday=6)
    inbound_df['weekday'] = inbound_df[col_dates].dt.weekday
    return inbound_df

inbound_2022_dry = get_weekdays(inbound_df=inbound_2022_dry, col_dates='Dates')

### Clean the data

removes all sundays and holidays in the complete dataset

In [0]:
shift_periods = [shift_period_1, shift_period_2, shift_period_3]
column_names = ['x_7days', 'x_14days', 'x_21days']

for i, period in enumerate(shift_periods):
    inbound_2022_dry[column_names[i]] = inbound_2022_dry['Dry Actuals'].shift(period)


In [0]:
inbound_2022_complete.head(50)

,Dates,Dry Actuals
0,2022-01-03,84224.000000
1,2022-01-04,49564.000000
2,2022-01-05,45890.000000
3,2022-01-06,26976.000000
4,2022-01-07,37044.000000
5,2022-01-08,56996.072927
6,2022-01-09,0.000000
7,2022-01-10,71540.000000
8,2022-01-11,63644.000000
9,2022-01-12,66433.000000


In [0]:
#drop Nan and sundays and holidays with Zero in Actuals 
inbound_2022_dry = inbound_2022_dry.dropna().loc[inbound_2022_dry['Dry Actuals'] != 0].reset_index(drop=True)


In [0]:
inbound_2022_dry.h


,Dates,Dry Actuals,x_7days,x_14days,x_21days,weekday
0,2022-01-24,102266.0,81534.000000,71540.0,84224.000000,0
1,2022-01-25,80584.0,72886.000000,63644.0,49564.000000,1
2,2022-01-26,70974.0,53783.000000,66433.0,45890.000000,2
3,2022-01-27,67565.0,0.000000,52802.0,26976.000000,3
4,2022-01-28,57571.0,52025.000000,43554.0,37044.000000,4
5,2022-01-29,61000.0,59776.587064,48456.0,56996.072927,5
6,2022-01-31,83232.0,102266.000000,81534.0,71540.000000,0
7,2022-02-01,80128.0,80584.000000,72886.0,63644.000000,1
8,2022-02-02,61760.0,70974.000000,53783.0,66433.000000,2
9,2022-02-03,62943.0,67565.000000,0.0,52802.000000,3


### Backtest

In [0]:
backtest_results = pd.DataFrame()

inbound_2022_train = inbound_2022_dry[['Dates', 'Dry Actuals']].copy()#original non filtered dataset, 


# Iterate through each backtesting period
while backtest_start_date + dt.timedelta(days=pred_period) <= backtest_end_date:
  print(f"Backtesting on: {backtest_start_date}")
  
  start_train_date = backtest_start_date
  # end_train_date = backtest_start_date + dt.timedelta(days=training_period)
  end_train_date = start_train_date + dt.timedelta(days=training_period)
  print(f"Backtesting training ending on: {end_train_date}") 

  lgbm_train = inbound_2022_train[(inbound_2022_train['Dates'] >= start_train_date) &(inbound_2022_train['Dates'] < end_train_date)].copy()
  print(f"print the shape:{lgbm_train.shape} ")
  # Convert Dates column to datetime
  lgbm_train['Dates'] = pd.to_datetime(lgbm_train['Dates'])
    
  #Train features 
  train_features_df = inbound_2022_dry[(lgbm_train['Dates'] >= start_train_date) &(inbound_2022_dry['Dates'] < end_train_date)].copy()
  features_values_train=train_features_df.iloc[:, -4:]

  # Filter the dates with values in Dry Actuals 
  features_train = features_values_train.copy() 
  target = lgbm_train['Dry Actuals']
  
  # Create LightGBM dataset
  lgb_train = lgb.Dataset(features_train, label=target)
    
  # Model training
  m = lgb.train(params, lgb_train)

  # Prediction for the current period
  future_dates = pd.date_range(start=end_train_date+dt.timedelta(days=15), periods=pred_period, freq='D')

  future_features = pd.DataFrame({'ds': future_dates})
  future_features['ds'] = pd.to_datetime(future_features['ds'])

  future_features = inbound_2022_dry[inbound_2022_dry['Dates'].isin(future_dates)].iloc[:, -3:].values ### Changed the source df

  prediction_dates = inbound_2022_dry[inbound_2022_dry['Dates'].isin(future_dates)]['Dates'] ### New variable to extract valid dates from the future date range
  
  #forecast using lgbm model m

  forecast = m.predict(future_features, num_iteration=m.best_iteration,predict_disable_shape_check=True)

  lgbm_pred = pd.DataFrame({'ds': prediction_dates, 'yhat': forecast}) ### Use dates form df rather than the original date range
    
  backtest_results = pd.concat([backtest_results, lgbm_pred])
    
  # Increase training period after each iterations 
  backtest_start_date += dt.timedelta(days=end_horizon)
 
print("Backtesting completed.")


Backtesting on: 2022-01-03 00:00:00
Backtesting training ending on: 2022-07-02 00:00:00
print the shape:(128, 2) 
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000058 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 129
[LightGBM] [Info] Number of data points in the train set: 128, number of used features: 4
[LightGBM] [Info] Start training from score 65732.078125
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

In [0]:
lgbm_train

,Dates,Dry Actuals
268,2022-12-19,76525.0
269,2022-12-20,75921.0
270,2022-12-21,58697.0
271,2022-12-22,62334.0
272,2022-12-23,55510.0
273,2022-12-24,37575.0
274,2022-12-26,90911.0
275,2022-12-27,63764.0
276,2022-12-28,47680.0
277,2022-12-29,39552.0


### Reformat backtest

In [0]:
backtest_results['Dates']= backtest_results['ds']
backtest_results['LGBM_FC']= backtest_results['yhat']
backtest_df = backtest_results[['Dates','LGBM_FC']].reset_index(drop=True).copy()
backtest_df['Dates'] = pd.to_datetime(backtest_df['Dates'])

### Join the backtest to the inbound

In [0]:
###Remove the data quality issues where FC is exactly te same as actuals

backtest_df_merge = pd.merge(inbound_2022_df[inbound_2022_df['Dry Fc']!=inbound_2022_df['Dry Actuals']],backtest_df, on="Dates", how='inner')[['Dates','Dry Actuals','Dry Fc','LGBM_FC']]
backtest_df_merge

,Dates,Dry Actuals,Dry Fc,LGBM_FC
0,2022-07-18,67408.0,76897.700,104541.911605
1,2022-07-19,51655.0,54842.880,103570.959140
2,2022-07-20,53350.0,51537.000,96828.883038
3,2022-07-22,45843.0,40934.138,101176.672585
4,2022-07-25,78763.0,79558.600,100299.036971
...,...,...,...,...
111,2022-12-26,90911.0,95488.000,98276.239288
112,2022-12-27,63764.0,71459.270,86527.281590
113,2022-12-28,47680.0,71916.320,89319.023783
114,2022-12-29,39552.0,60346.346,76625.647127


## Evaluate

### Create error columns

In [0]:
for error in  error_to_calc:
  for i in range(len(actual_cols)):
    calculate_errors(df_eval= backtest_df_merge, actuals_col=actual_cols[i], fc_col= fc_cols[i],error_to_calculate=error,error_colname_suffix=error_suffixes[i])
    

In [0]:
backtest_df_merge

,Dates,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,Outside_range_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_LGBM
0,2022-07-18,67408.0,76897.700,104541.911605,9489.700,37133.911605,9489.700,37133.911605,9.005441e+07,1.378927e+09,0.140780,True,0.550883,True
1,2022-07-19,51655.0,54842.880,103570.959140,3187.880,51915.959140,3187.880,51915.959140,1.016258e+07,2.695267e+09,0.061715,False,1.005052,True
2,2022-07-20,53350.0,51537.000,96828.883038,-1813.000,43478.883038,1813.000,43478.883038,3.286969e+06,1.890413e+09,0.033983,False,0.814974,True
3,2022-07-22,45843.0,40934.138,101176.672585,-4908.862,55333.672585,4908.862,55333.672585,2.409693e+07,3.061815e+09,0.107080,True,1.207026,True
4,2022-07-25,78763.0,79558.600,100299.036971,795.600,21536.036971,795.600,21536.036971,6.329794e+05,4.638009e+08,0.010101,False,0.273428,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2022-12-26,90911.0,95488.000,98276.239288,4577.000,7365.239288,4577.000,7365.239288,2.094893e+07,5.424675e+07,0.050346,False,0.081016,False
112,2022-12-27,63764.0,71459.270,86527.281590,7695.270,22763.281590,7695.270,22763.281590,5.921718e+07,5.181670e+08,0.120684,True,0.356993,True
113,2022-12-28,47680.0,71916.320,89319.023783,24236.320,41639.023783,24236.320,41639.023783,5.873992e+08,1.733808e+09,0.508312,True,0.873302,True
114,2022-12-29,39552.0,60346.346,76625.647127,20794.346,37073.647127,20794.346,37073.647127,4.324048e+08,1.374455e+09,0.525747,True,0.937339,True


### Calculate global metrics

In [0]:
global_metrics_dry = describe_inbound_fc(inbound_fc=backtest_df_merge, cols_to_keep=['Dry Actuals','Dry Fc','LGBM_FC','Bias_Dry_WOW','Bias_Dry_LGBM', 'AbsError_Dry_WOW','AbsError_Dry_LGBM','SqError_Dry_WOW','SqError_Dry_LGBM','RelAbsError_Dry_WOW','RelAbsError_Dry_LGBM', 'Outside_range_Dry_WOW', 'Outside_range_Dry_LGBM'])

<command-203330119889549>:25: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  described_df = inbound_fc.describe(include='all')[cols_to_keep].rename_axis('Metrics').reset_index(drop=False).copy()


In [0]:
global_metrics_dry

,Metrics,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_WOW,Outside_range_Dry_LGBM
0,count,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,1.160000e+02,1.160000e+02,116.000000,116.000000,116,116
1,unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
2,top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
3,freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77,103
4,first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,mean,68658.474138,70432.331293,91204.101842,1773.857155,22545.627704,6145.002896,29707.787215,1.013758e+08,1.192755e+09,0.110535,0.533495,NaN,NaN
7,std,26707.112555,24551.818188,7935.723404,9954.063606,26275.487711,8010.486584,17688.969375,4.128963e+08,1.315576e+09,0.234379,0.465367,NaN,NaN
8,min,21742.000000,35712.572000,75959.631893,-59204.840000,-93868.117739,4.000000,557.809348,1.600000e+01,3.111513e+05,0.000081,0.005412,NaN,NaN
9,25%,51657.250000,54152.369000,84898.314515,-2506.930000,12732.457469,2199.899500,16613.519993,4.839992e+06,2.760191e+08,0.029886,0.236444,NaN,NaN


### Visualize profiles

In [0]:
plot_two_inbound_fc(inbound_df=backtest_df_merge, actuals_col='Dry Actuals', fc_col_1='Dry Fc', fc_col_2= 'LGBM_FC', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

### Visualize errors

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='Bias_Dry_WOW', error_col_2= 'Bias_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='AbsError_Dry_WOW', error_col_2= 'AbsError_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')